## Renewable energy sources optimization

### Problem formulation

We need to optimize the distribution of energy from multiple sources to different consumers to minimize the overall cost, while satisfying demand constraints.

### Parameters

- **Energy Sources**: Two sources (e.g., solar and wind), each with different capacities and costs.
- **Consumers**: Two consumers, each with a specific energy demand.
- **Objective**: Minimize the total cost of energy distribution while meeting all consumer demands without exceeding the source capacities.

**Source Capacities**:
- Solar = 100 units
- Wind = 150 units

**Source Costs**:
- Solar = 3$ / unit
- Wind = 2$ / unit

**Consumer Demands**:
- Consumer A = 90 units
- Consumer B = 120 units

### Variables

- \( x_{i,j} \): Amount of energy transferred from source \( i \) to consumer \( j \).

### Objective

Minimize total cost:
\[ \text{Cost} = 3x_{solar,A} + 3x_{solar,B} + 2x_{wind,A} + 2x_{wind,B} \]

### Constraints

1. Total energy supplied to each consumer meets their demand.
2. The total energy taken from each source does not exceed its capacity.

### Mathematical Formulation

**Objective Function**:
\[ \min \; 3x_{solar,A} + 3x_{solar,B} + 2x_{wind,A} + 2x_{wind,B} \]

**Constraints**:
- \( x_{solar,A} + x_{solar,B} \leq 100 \) (Solar capacity)
- \( x_{wind,A} + x_{wind,B} \leq 150 \) (Wind capacity)
- \( x_{solar,A} + x_{wind,A} = 90 \) (Demand of Consumer A)
- \( x_{solar,B} + x_{wind,B} = 120 \) (Demand of Consumer B)
- \( x_{i,j} \geq 0 \) for all \( i, j \)

### TensorFlow Implementation

We will use TensorFlow to approximate a solution via a gradient descent approach, treating constraints as penalty terms in the loss function.

In [1]:
import tensorflow as tf

# Initialize variables
x_solar_A = tf.Variable(50.0, trainable=True)  # Start with arbitrary feasible values
x_solar_B = tf.Variable(50.0, trainable=True)
x_wind_A = tf.Variable(40.0, trainable=True)
x_wind_B = tf.Variable(80.0, trainable=True)

# Define constraints as penalties
def capacity_constraints():
    return [
        100 - (x_solar_A + x_solar_B),  # Solar capacity
        150 - (x_wind_A + x_wind_B),    # Wind capacity
        x_solar_A + x_wind_A - 90,      # Demand of Consumer A
        x_solar_B + x_wind_B - 120     # Demand of Consumer B
    ]

# Cost function
cost = 3*x_solar_A + 3*x_solar_B + 2*x_wind_A + 2*x_wind_B

# Optimizer
optimizer = tf.optimizers.SGD(learning_rate=0.01)

# Training step
def train_step():
    with tf.GradientTape() as tape:
        constraints = capacity_constraints()
        penalty = tf.reduce_sum(tf.square(constraints))  # Squared penalties for constraint violation
        loss = cost + penalty
    gradients = tape.gradient(loss, [x_solar_A, x_solar_B, x_wind_A, x_wind_B])
    optimizer.apply_gradients(zip(gradients, [x_solar_A, x_solar_B, x_wind_A, x_wind_B]))

# Optimization loop
for _ in range(1000):
    train_step()

# Output the results
print(f'x_solar_A: {x_solar_A.numpy()}, x_solar_B: {x_solar_B.numpy()}')
print(f'x_wind_A: {x_wind_A.numpy()}, x_wind_B: {x_wind_B.numpy()}')


2024-04-25 16:40:16.123392: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 16:40:16.128033: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 16:40:16.175931: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 16:40:17.190311: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


x_solar_A: 47.50000762939453, x_solar_B: 42.50007629394531
x_wind_A: 52.5, x_wind_B: 87.4999008178711
